In [1]:
import math
import itertools

from experiment import *

from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

tf.get_logger().setLevel("ERROR")

# tf.config.set_visible_devices([], "GPU")

2022-07-03 17:59:31.340918: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-03 17:59:31.363545: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-03 17:59:31.364022: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
base_experiment_parameters = {
    "vocab_size": 3000,
    "sequence_length": 300,
    "hidden_layers": [3000, 1500],
    "embedding_dim": 2,
    "dropout_rate": 0.5,
}

experiment_parameters = {
    "vocab_size": 5168,
    "sequence_length": 515,
    "hidden_layers": [5168, 2584],
    "embedding_dim": 2,
    "dropout_rate": 0.5,
}

In [ ]:
experiment(**base_experiment_parameters, verbose=1)

In [ ]:
experiment(**experiment_parameters, verbose=1)

In [4]:
#results, max = next_step(vocab_size=2000, test_run=True)
next_step(**experiment_parameters, test_run=True)

Next Step: vocab_size=5168 sequence_length=515 hidden_layers=[5168, 2584] embedding_dim=2 dropout_rate=0.5 epochs=5 verbose=1 exploration_rate=0.1
trial_vocab_size=5688 trial_sequence_length=566 trial_hidden_layers=[5688, 2844, 1422]


In [ ]:
[(key, val) for key, val in results.items() if val > .891]

In [ ]:
#for _ in range(10):
#    (
#        next_sequence_length,
#        next_vocab_size,
#        next_hidden_units,
#        next_embeddings,
#    ) = next_step(sequence_length, vocab_size, hidden_units, embeddings, 1.25)

#    if (
#        next_sequence_length == sequence_length
#        and next_vocab_size == vocab_size
#        and next_hidden_units == hidden_units
#        and next_embeddings == embeddings
#    ):
#        break

#    sequence_length = next_sequence_length
#    vocab_size = next_vocab_size
#    hidden_units = next_hidden_units
#    embeddings = next_embeddings